In [1]:
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import classification_report, accuracy_score, precision_score

tf.__version__

'2.4.1'

In [2]:
# -------- TEST USER ----------- #

TEST_USER      = '002'

BASE_DIR       = '../'
IMG_DIR        = 'BW-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 160
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 7
LEARNING_RATE  = 0.001
DECAY          = 0.0

CONFIG         = '_L_7_S_160x160_E_7'

XY_WEIGHTS     = np.array([0.91, 0.75, 0.61, 0.63, 0.51, 0.66, 0.81, 0.65, 0.65, 0.31,
                           0.66, 0.29, 0.34, 0.64, 0.64, 0.31])
YZ_WEIGHTS     = np.array([0.73, 0.71, 0.70, 0.79, 0.76, 0.38, 0.80, 0.61, 0.58, 0.73,
                           0.49, 0.26, 0.26, 0.52, 0.59, 0.54])
ZX_WEIGHTS     = np.array([0.33, 0.66, 0.51, 0.54, 0.37, 0.51, 0.71, 0.30, 0.75, 0.41,
                           0.40, 0.27, 0.24, 0.61, 0.36, 0.49])

In [3]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

In [4]:
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

loading data for j gesture on the XY plane ... √
loading data for z gesture on the XY plane ... √
loading data for bad gesture on the XY plane ... √
loading data for deaf gesture on the XY plane ... √
loading data for fine gesture on the XY plane ... √
loading data for good gesture on the XY plane ... √
loading data for goodbye gesture on the XY plane ... √
loading data for hello gesture on the XY plane ... √
loading data for hungry gesture on the XY plane ... √
loading data for me gesture on the XY plane ... √
loading data for no gesture on the XY plane ... √
loading data for please gesture on the XY plane ... √
loading data for sorry gesture on the XY plane ... √
loading data for thankyou gesture on the XY plane ... √
loading data for yes gesture on the XY plane ... √
loading data for you gesture on the XY plane ... √
loading data for j gesture on the YZ plane ... √
loading data for z gesture on the YZ plane ... √
loading data for bad gesture on the YZ plane ... √
loading data for de

In [5]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [6]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(GESTURES))

In [8]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE, decay=DECAY),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [9]:
model_xy = get_model()
history_xy = model_xy.fit(X_train_xy, y_train_xy, validation_data=(X_test_xy, y_test_xy), epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 18s 486ms/step - loss: 2.8148 - accuracy: 0.1479 - val_loss: 1.8696 - val_accuracy: 0.4875
Epoch 2/7
30/30 [==============================] - 13s 438ms/step - loss: 1.6636 - accuracy: 0.5047 - val_loss: 1.5398 - val_accuracy: 0.5312
Epoch 3/7
30/30 [==============================] - 13s 432ms/step - loss: 1.2321 - accuracy: 0.6330 - val_loss: 1.4295 - val_accuracy: 0.5750
Epoch 4/7
30/30 [==============================] - 13s 435ms/step - loss: 0.9927 - accuracy: 0.6895 - val_loss: 1.3126 - val_accuracy: 0.5688
Epoch 5/7
30/30 [==============================] - 13s 433ms/step - loss: 0.8375 - accuracy: 0.7484 - val_loss: 1.2132 - val_accuracy: 0.6438
Epoch 6/7
30/30 [==============================] - 13s 438ms/step - loss: 0.7414 - accuracy: 0.7813 - val_loss: 1.1426 - val_accuracy: 0.6438
Epoch 7/7
30/30 [==============================] - 13s 420ms/step - loss: 0.6315 - accuracy: 0.8290 - val_loss: 1.1660 - val_accuracy: 0.6687


In [10]:
# prob_xy = tf.keras.Sequential([model_xy, tf.keras.layers.Softmax()])
# y_pred_xy = prob_xy.predict(X_test_xy)
y_pred_xy = model_xy.predict(X_test_xy)
y_pred = np.argmax(y_pred_xy, axis=1)
print(classification_report(y_test_xy.ravel(), y_pred, zero_division=0))
prc_xy = precision_score(y_test_xy.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95        10
         1.0       0.88      0.70      0.78        10
         2.0       0.89      0.80      0.84        10
         3.0       0.62      1.00      0.77        10
         4.0       1.00      0.90      0.95        10
         5.0       0.91      1.00      0.95        10
         6.0       1.00      0.20      0.33        10
         7.0       1.00      0.70      0.82        10
         8.0       0.77      1.00      0.87        10
         9.0       0.83      0.50      0.62        10
        10.0       0.86      0.60      0.71        10
        11.0       0.17      0.10      0.12        10
        12.0       0.35      0.70      0.47        10
        13.0       0.44      0.70      0.54        10
        14.0       0.33      0.20      0.25        10
        15.0       0.46      0.60      0.52        10

    accuracy                           0.67       160
   macro avg       0.71   

In [11]:
model_yz = get_model()
history_yz = model_yz.fit(X_train_yz, y_train_yz, validation_data=(X_test_yz, y_test_yz), epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 15s 427ms/step - loss: 3.4494 - accuracy: 0.1546 - val_loss: 2.3939 - val_accuracy: 0.2937
Epoch 2/7
30/30 [==============================] - 12s 407ms/step - loss: 1.9281 - accuracy: 0.4067 - val_loss: 1.6357 - val_accuracy: 0.4563
Epoch 3/7
30/30 [==============================] - 13s 422ms/step - loss: 1.3018 - accuracy: 0.5872 - val_loss: 1.2887 - val_accuracy: 0.5938
Epoch 4/7
30/30 [==============================] - 13s 427ms/step - loss: 0.9571 - accuracy: 0.6678 - val_loss: 1.1449 - val_accuracy: 0.6250
Epoch 5/7
30/30 [==============================] - 12s 395ms/step - loss: 0.8095 - accuracy: 0.7652 - val_loss: 1.0476 - val_accuracy: 0.6625
Epoch 6/7
30/30 [==============================] - 14s 465ms/step - loss: 0.6654 - accuracy: 0.8025 - val_loss: 0.9730 - val_accuracy: 0.6875
Epoch 7/7
30/30 [==============================] - 15s 505ms/step - loss: 0.6119 - accuracy: 0.8085 - val_loss: 0.9529 - val_accuracy: 0.6812


In [12]:
# prob_yz = tf.keras.Sequential([model_yz, tf.keras.layers.Softmax()])
# y_pred_yz = prob_yz.predict(X_test_yz)
y_pred_yz = model_yz.predict(X_test_yz)
y_pred = np.argmax(y_pred_yz, axis=1)
print(classification_report(y_test_yz.ravel(), y_pred, zero_division=0))
prc_yz = precision_score(y_test_yz.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

              precision    recall  f1-score   support

         0.0       0.82      0.90      0.86        10
         1.0       1.00      0.90      0.95        10
         2.0       1.00      0.70      0.82        10
         3.0       1.00      0.80      0.89        10
         4.0       0.80      0.80      0.80        10
         5.0       0.62      1.00      0.77        10
         6.0       1.00      0.80      0.89        10
         7.0       1.00      0.50      0.67        10
         8.0       0.67      0.40      0.50        10
         9.0       0.53      1.00      0.69        10
        10.0       0.50      0.40      0.44        10
        11.0       0.38      0.30      0.33        10
        12.0       0.33      0.80      0.47        10
        13.0       1.00      1.00      1.00        10
        14.0       0.33      0.10      0.15        10
        15.0       0.62      0.50      0.56        10

    accuracy                           0.68       160
   macro avg       0.73   

In [13]:
model_zx = get_model()
history_zx = model_zx.fit(X_train_zx, y_train_zx, validation_data=(X_test_zx, y_test_zx), epochs=EPOCHS)

Epoch 1/7
30/30 [==============================] - 17s 466ms/step - loss: 3.4401 - accuracy: 0.1980 - val_loss: 2.9293 - val_accuracy: 0.2438
Epoch 2/7
30/30 [==============================] - 14s 485ms/step - loss: 2.2665 - accuracy: 0.3358 - val_loss: 2.2706 - val_accuracy: 0.3375
Epoch 3/7
30/30 [==============================] - 14s 477ms/step - loss: 1.6759 - accuracy: 0.4790 - val_loss: 2.0179 - val_accuracy: 0.4563
Epoch 4/7
30/30 [==============================] - 15s 504ms/step - loss: 1.2976 - accuracy: 0.5732 - val_loss: 1.8477 - val_accuracy: 0.4563
Epoch 5/7
30/30 [==============================] - 12s 412ms/step - loss: 1.0752 - accuracy: 0.6667 - val_loss: 1.7841 - val_accuracy: 0.4750
Epoch 6/7
30/30 [==============================] - 14s 465ms/step - loss: 0.9180 - accuracy: 0.7193 - val_loss: 1.7819 - val_accuracy: 0.5000
Epoch 7/7
30/30 [==============================] - 15s 494ms/step - loss: 0.8254 - accuracy: 0.7494 - val_loss: 1.7412 - val_accuracy: 0.5125


In [14]:
# prob_zx = tf.keras.Sequential([model_zx, tf.keras.layers.Softmax()])
# y_pred_zx = prob_zx.predict(X_test_zx)
y_pred_zx = model_zx.predict(X_test_zx)
y_pred = np.argmax(y_pred_zx, axis=1)
print(classification_report(y_test_zx.ravel(), y_pred, zero_division=0))
prc_zx = precision_score(y_test_zx.ravel(), y_pred, zero_division=0, average=None)
tf.keras.backend.clear_session()

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        10
         1.0       0.33      0.10      0.15        10
         2.0       0.60      0.30      0.40        10
         3.0       0.77      1.00      0.87        10
         4.0       0.27      0.30      0.29        10
         5.0       0.42      1.00      0.59        10
         6.0       1.00      0.90      0.95        10
         7.0       0.43      0.60      0.50        10
         8.0       0.83      1.00      0.91        10
         9.0       0.50      0.60      0.55        10
        10.0       0.09      0.10      0.10        10
        11.0       0.00      0.00      0.00        10
        12.0       0.40      0.20      0.27        10
        13.0       0.91      1.00      0.95        10
        14.0       0.06      0.10      0.07        10
        15.0       1.00      1.00      1.00        10

    accuracy                           0.51       160
   macro avg       0.48   

In [15]:
y_total = y_pred_xy + y_pred_yz + y_pred_zx
y_pred = np.argmax(y_total, axis=1)
report = classification_report(y_test_xy.ravel(), y_pred, zero_division=0)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      0.90      0.95        10
         1.0       1.00      1.00      1.00        10
         2.0       1.00      1.00      1.00        10
         3.0       0.91      1.00      0.95        10
         4.0       0.91      1.00      0.95        10
         5.0       0.77      1.00      0.87        10
         6.0       1.00      1.00      1.00        10
         7.0       1.00      1.00      1.00        10
         8.0       1.00      1.00      1.00        10
         9.0       1.00      1.00      1.00        10
        10.0       1.00      0.60      0.75        10
        11.0       0.33      0.10      0.15        10
        12.0       0.36      0.90      0.51        10
        13.0       1.00      1.00      1.00        10
        14.0       1.00      0.10      0.18        10
        15.0       0.91      1.00      0.95        10

    accuracy                           0.85       160
   macro avg       0.89   

In [16]:
config = '\n\nTEST_USER ' + TEST_USER + ' T: ' + str(int(time.time())) + '\n'
underline = '=====================================\n'
log_dir = os.path.join(BASE_DIR, LOG_DIR)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
f = open(os.path.join(log_dir, 'logs_sptl_bw' + CONFIG + '.txt'), 'a')
f.write(config)
f.write(underline)
f.write(report)
f.close()

In [17]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_xy' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_xy, precision=2, max_line_width=100) + '\n')
# f.close()

In [18]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_yz' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_yz, precision=2, max_line_width=100) + '\n')
# f.close()

In [19]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_zx' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_zx, precision=2, max_line_width=100) + '\n')
# f.close()